<a href="https://www.kaggle.com/code/aleksandrmorozov123/damo-yolo?scriptVersionId=216234329" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**DAMO-YOLO: a fast and accurate object detection method with some new techs, including NAS backbones, efficient RepGFPN, ZeroHead, AlignedOTA, and distillation enhancement**

In [ ]:
!git clone https://github.com/tinyvision/damo-yolo.git

In [ ]:
!pip install --upgrade gdown
!pip install -r requirements.txt
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install moviepy==0.2.3.5 imageio==2.4.1

In [ ]:
from IPython.display import Image

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

In [ ]:
!wget -c https://www.pakutaso.com/shared/img/thumb/goriueseii-_28A5611-8954_TP_V4.jpg \
      -O ./assets/test_01.jpg

In [ ]:
!wget -c https://raw.githubusercontent.com/timmeinhardt/trackformer/main/data/snakeboard/snakeboard.mp4 \
      -O ./assets/snakeboard.mp4

In [ ]:
!mkdir pretrained_models

!gdown https://drive.google.com/uc?id=1-O-ObHN970GRVKkL1TiAxfoMCpYGJS6B \
  -O ./pretrained_models/damoyolo_tinynasL25_S.pth

**Object detection on image**

In [ ]:
!cp ./damo-yolo/tools/demo.py ./demo.py

!python demo.py \
  -f ./configs/damoyolo_tinynasL25_S.py \
  --engine ./pretrained_models/damoyolo_tinynasL25_S.pth \
  --engine_type torch \
  --conf 0.6 \
  --infer_size 640 640 \
  --device cuda \
  --path ./assets/test_01.jpg

In [ ]:
Image('./assets/test_01.jpg')